# SQL databases

There exist different types of database management systems, **depending on the type of data** they store - for example relational, object, document, XML, noSQL, etc. In this section we focus on the relational databases which provide a common interface using the SQL query language – a common basic query language, which we use for searching and manipulating data in relational databases. Example database management systems are Oracle, IBM DB2, MSSQL, MySQL, PostgreSQL or SQLite. SQL databases have grounds in relational calculus or relational algebra theory. An example of relation representation (relation != table):

![](relation.png)

## SQL introduction

Structure Query Language (SQL), /sequel/, is a declarative language, standardized in 1986. Each RDBMS may implement their own variant of SQL flavour. By default, we can organize SQL in groups of statemens, such as DDL, DML, DCL, TCL. We will focus on DDL and DML. 

Column data types (ANSI SQL):

| Type | SQL data types |
|------|----------------|
| binary | BIT, BIT VARYING |
| time | DATE, TIME, TIMESTAMP |
| number | DECIMAL, REAL, DOUBLE PRECISION, FLOAT, SMALLINT, INTEGER, INTERVAL |
| text | CHARACTER, CHARACTER VARYING, NATIONAL CHARACTER, NATIONAL CHARACTER VARYING |

**Databases / Schemas**

```sql
CREATE DATABASE DatabaseName

USE DatabaseName --or addressing objects like DatabaseName.ObjectName
```

**Tables**

```sql
CREATE TABLE TableName (
    {colName dataType [NOT NULL] [UNIQUE]
    [DEFAULT defaultOption]
    [CHECK searchCondition] [,...]}
    [PRIMARY KEY (listOfColumns),]
    {[UNIQUE (listOfColumns),] [...,]}
    {[FOREIGN KEY (listOfFKColumns)
       REFERENCES ParentTableName [(listOfCKColumns)],
       [ON UPDATE referentialAction]
       [ON DELETE referentialAction ]] [,...]}
    {[CHECK (searchCondition)]}) 
```

Constraints (data correctness):
```sql
ALTER TABLE TableName DROP CONSTRAINT fkName;
```

Schema changes:
```sql
ALTER TABLE TableName ADD NewColumnName INTEGER NOT NULL DEFAULT 0;
```

Indexes:
```sql
CREATE [UNIQUE|FULLTEXT|SPATIAL] INDEX index_name
   ON table_name
      (column1 [ASC|DESC],
       column2 [ASC|DESC],
...);
```

Removing table:
```sql
DROP TABLE TableName [RESTRICT | CASCADE]
```

**Inserting**
```sql
INSERT INTO TableName [ (columnList) ] VALUES
  (dataValueList)[, (dataValueList), ...]
```

**Updating**
```sql
UPDATE TableName
SET columnName1 = dataValue1
   [, columnName2 = dataValue2...]
[WHERE searchCondition]
```

**Deleting**
```sql
DELETE FROM TableName
[WHERE searchCondition]
```

**Querying**

* Nested SELECTs
* Connecting multiple tables - JOINs
* Comparisons - equality (>, <, =, <=, >=, BETWEEN), text comparisons (LIKE), set comparisons (IN, ALL, ANY), null comparisons (IS NULL, IS NOT NULL) or RDBMS specific functions.
* Aggregate functions (e.g., COUNT, SUM, AVG, MIN, MAX, GROUP_CONCAT) for groups

```sql
SELECT [DISTINCT | ALL] {* | [columnExpression [AS newName]] [,...] }
FROM 
  TableName [alias] [, ...] 
[WHERE condition]
[GROUP BY columnList] 
[HAVING condition]
[ORDER BY columnList [ASC|DESC]]
```

## SQLite

[SQLite](https://www.sqlite.org/download.html) is an open source, zero-configuration, self-contained, stand-alone, transaction relational database engine designed to be embedded into an application. Good source of information about the use of the database is available at [http://www.sqlitetutorial.net/](http://www.sqlitetutorial.net/).

Use command as `sqlite3` or `sqlite3 itds_course.db`.

![](model.png)

By default, referential integrity checks are not enforced. Checks can be enabled (session-based):

```sql
-- Set the foreign keys checking on
PRAGMA foreign_keys = ON;
-- Check the foreign keys setting (1=ON, 0=OFF)
PRAGMA foreign_keys;
```

**Tables creation**

* Naming
    
```sql
-- Creating Branch table
CREATE TABLE Branch (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    address TEXT NOT NULL,
    telephone TEXT
);

-- Employee
CREATE TABLE Employee (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    id_branch INTEGER NOT NULL,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    employment_start TEXT NOT NULL,
    employment_end TEXT,
    FOREIGN KEY (id_branch) REFERENCES Branch(id)
);

-- Salary
CREATE TABLE Salary (
    year INTEGER,
    month INTEGER,
    id_employee INTEGER,
    amount REAL NOT NULL,
    PRIMARY KEY (year, month, id_employee),
    FOREIGN KEY (id_employee) REFERENCES Employee(id)
);
```

**Data insertion**

```sql
-- Branches
INSERT INTO Branch (name, address, telephone) VALUES 
    ("Grosuplje Fried Chicken", "Kolodvorska 1, Grosuplje", "01/786-42-31");
INSERT INTO Branch (name, address) VALUES 
    ("Hot Pork Wings", "Cesta v Kranj 35, Ljubljana");

-- Employees (still employed)
INSERT INTO Employee (id_branch, first_name, last_name, employment_start) VALUES
    (1, "Marko", "Novaković", "1972-01-25"),
    (1, "Jože", "Novak", "1995-08-22"),
    (1, "Janez", "Gruden", "1999-12-07"),
    (1, "Tine", "Stegel", "2002-04-28"),
    (1, "Nina", "Gec", "2010-03-08"),
    (1, "Petar", "Stankovski", "2018-10-01");

-- Employees (retired)
INSERT INTO Employee (id_branch, first_name, last_name, employment_start, employment_end) VALUES
    (1, "Jana", "Žitnik", "1987-06-07", "1987-06-12"),
    (1, "Patricio", "Rozman", "2010-04-22", "2022-01-01"),
    (1, "Robert", "Demšar", "1996-12-06", "1997-07-02");

-- Salaries
INSERT INTO Salary (year, month, id_employee, amount) VALUES 
    (2010, 05, 2, 950.37),
    (2010, 06, 2, 955.2),
    (2010, 07, 2, 990.49),
    (2010, 08, 2, 1200.5),
    (2010, 05, 6, 980.43),
    (2010, 05, 7, 1320.2),
    (2010, 05, 8, 2100.67),
    (2010, 06, 8, 2140.99),
    (2010, 07, 8, 2099.11);
```

**Selecting**

```sql
-- Selecting all columns
SELECT * FROM Branch;

-- Selecting a column with a condition
SELECT employment_start 
FROM Employee 
WHERE last_name = "Gec";

-- Result value concatenation
SELECT id, first_name || ' ' || last_name AS name 
FROM Employee 
WHERE employment_end IS NULL;

-- Grouping and aggregations
SELECT b.id, b.name, COUNT(e.id), MAX(s.amount)
FROM Branch b, Employee e, Salary s
WHERE
    e.id_branch = b.id AND  
    s.id_employee = e.id
GROUP BY b.id, b.name;

-- Grouping and outer joins
SELECT b.id, b.name, COUNT(e.id), MAX(s.amount)
FROM Branch b LEFT OUTER JOIN Employee e ON e.id_branch = b.id
     LEFT OUTER JOIN Salary s ON s.id_employee = e.id
GROUP BY b.id, b.name;

-- Nested select statement
SELECT * 
FROM (SELECT e.id, e.first_name, e.last_name, s.amount, e.employment_end, s.year, s.month
      FROM Employee e, Salary s
      WHERE
         e.id = s.id_employee AND
         e.employment_end IS NOT NULL) t
WHERE
    strftime('%Y', t.employment_end) < t.year OR
    (strftime('%Y', t.employment_end) = t.year AND
     strftime('%m', t.employment_end) < t.month);

-- Storing SELECT results into a table
CREATE TABLE SalaryAnalysis AS
SELECT e.id, e.employment_start, e.employment_end, 
       s.year || '-' || s.month || '-01' AS month, s.amount 
FROM Employee e INNER JOIN Salary s ON e.id = s.id_employee;

SELECT * FROM SalaryAnalysis;

-- Views
CREATE VIEW SalaryAnalysisView (id, start, end, month, amount) AS
SELECT e.id, e.employment_start, e.employment_end, 
       s.year || '-' || s.month || '-01' AS month, s.amount 
FROM Employee e INNER JOIN Salary s ON e.id = s.id_employee;

SELECT * FROM SalaryAnalysisView;
```

**Updating and deleting**

```sql
-- Set new phone number to the Hot Pork Wings branch
UPDATE Branch
 SET telephone = "05/869-22-54"
 WHERE name = "Hot Pork Wings";

-- Retire the Employee 3 starting today
UPDATE Employee
 SET employment_end = date('now')
 WHERE id = 3;
 
-- Deleting employees that are not employed anymore
DELETE FROM Employee WHERE employment_end IS NOT NULL;

DELETE FROM Salary WHERE id_employee IN (
    SELECT id FROM Employee WHERE employment_end IS NOT NULL
);
```

## SQLite and Python

To use SQLite from Python, please install the *sqlite3* dependency. If you use Anaconda Python distribution, issue command:

```
conda install -c anaconda sqlite
```

Also check the [documentation of the sqlite3 package](https://docs.python.org/3/library/sqlite3.html) which we are using from Python.

In [1]:
import sqlite3
#Creates a new db if it does not exist
conn = sqlite3.connect('itds_course.db') 

### Creating tables

Let's the strucure we introduced above.

In [2]:
# Read file
create_insert_statements = open('create_and_populate_SQLite.sql', 'r').read()

c = conn.cursor()

c.executescript(create_insert_statements)

# Save (commit) the changes
conn.commit()

### Selecting data

Let's check, what is in the database.

In [3]:
cursor = c.execute('''
    SELECT b.id, b.name, COUNT(e.id), MAX(s.amount)
    FROM Branch b LEFT OUTER JOIN Employee e ON e.id_branch = b.id
         LEFT OUTER JOIN Salary s ON s.id_employee = e.id
    GROUP BY b.id, b.name;
''')

for row in cursor:
    print(f"\tBranch id: {row[0]}\n\t\tBranch name: '{row[1]}'\n\t\tEmployee number: \
    {row[2]}\n\t\tMaximum salary: {row[3]}")

# You should close the connection when stopped using the database.
conn.close()

	Branch id: 1
		Branch name: 'Grosuplje's Fried Chicken'
		Employee number:     14
		Maximum salary: 2140.99
	Branch id: 2
		Branch name: 'Hot Pork Wings'
		Employee number:     0
		Maximum salary: None


In [4]:
# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

## Standalone SQLite tools

Nice and lightweight GUI software to create, open or manage SQLite databases is [DB Browser for SQLite](https://sqlitebrowser.org/). It is available for multiple operating systems.

![Screenshot](screenshot.png)

## PostgreSQL and Python

In some cases we need more powerful relational database features. In this case we propose to use a more advanced relational database management system, such as PostgreSQL, MS SQL, MySQL or others.

Let's use an official [PostgreSQL database Docker image](https://hub.docker.com/_/postgres) and run it:

```bash
docker run --name pg-server \
    -e POSTGRES_PASSWORD=BeReSeMi.DataScience \
    -e POSTGRES_USER=user \
    -p 5432:5432 \
    -d --rm postgres:12.2
```

We can also directly connect to the database using the following command:

```bash
docker exec -it pg-server psql -U user -W
```

First we need to create a database, called `courseitds` and connect to it:

```
user=# create database courseitds;
CREATE DATABASE
user=# \c courseitds
Password for user user:
You are now connected to database "courseitds" as user "user".
```

To use PostgreSQL from Python, please install the *psycopg2* dependency. If you use Anaconda Python distribution, issue command:

```bash
conda install -c anaconda psycopg2
```

Also check the [documentation of the package](https://www.psycopg.org/docs/) which we are using from Python.

In [5]:
import psycopg2
conn = psycopg2.connect(host="localhost", 
                        dbname="courseitds",
                        user="user", 
                        password="BeReSeMi.DataScience")
conn.autocommit = True

### Creating tables

Let's the strucure we introduced above.

In [6]:
# Read file
create_insert_statements = open('create_and_populate_PostgreSQL.sql', 'r').read()

cur = conn.cursor()
cur.execute(create_insert_statements)

### Selecting data

Let's check, what is in the database.

In [7]:
cursor = cur.execute('''
    SELECT b.id, b.name, COUNT(e.id), MAX(s.amount)
    FROM Branch b LEFT OUTER JOIN Employee e ON e.id_branch = b.id
         LEFT OUTER JOIN Salary s ON s.id_employee = e.id
    GROUP BY b.id, b.name;
''')

for branch_id, branch_name, emp_num, max_salary in cur.fetchall():
    print(f"\tBranch id: {branch_id}\n\t\tBranch name: '{branch_name}'\n\t\tEmployee number: \
    {emp_num}\n\t\tMaximum salary: {max_salary}")

	Branch id: 2
		Branch name: 'Hot Pork Wings'
		Employee number:     0
		Maximum salary: None
	Branch id: 1
		Branch name: 'Grosuplje's Fried Chicken'
		Employee number:     14
		Maximum salary: 2140.99


In [8]:
cur.close()
# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

## Standalone SQL tools

There are a number of different GUI-based clients for RDBMSs. JetBrains DataGrip is one of them

![](datagrip.png)